In [2]:
import os
from shutil import copyfile
from pathlib import Path
import pandas as pd

In [3]:
centroids = pd.read_csv('centroids.csv')
cities = centroids.city

In [4]:
def create_folder(name):
    try:
        os.mkdir(name)
    except FileExistsError:
        pass

### Share maps

In [4]:
ssps = ['1', '2', '3']

In [7]:
map_folders = ['air', 'AOI', 'landslide', 'summer', 'fluvial', 'pluvial']
for s in ssps:
    for f in ['fu', 'pu']:
        for y in ['2050', '2100']:
            map_folders.append('bu_proj_' + f + '_ssp' + s + '_' + y)
    for t in ['day', 'night']:
        map_folders.append('summer_' + t + '_ssp' + s)
for y in ['2050', '2100']:
    for s in ['', '_RL10']:
        map_folders.append('slr' + s + '_' + y)

for i in map_folders:
    create_folder(Path('maps') / i)

In [8]:
def copy_file(city, orig_suffix, new_folder = '', new_suffix = ''):
    city1 = city.lower().replace(' ', '_')
    
    if new_folder == '':
        new_folder = orig_suffix
    if new_suffix == '':
        new_suffix = orig_suffix
    
    try:
        copyfile(city / Path('maps') / (city1 + '_' + orig_suffix + '.png'), 
                 'maps' / Path(new_folder) / (city1 + '_' + new_suffix + '.png'))
    except FileNotFoundError:
        pass

In [10]:
for city in cities:
    copy_file(city, 'air_fixedlabel_2019', 'air', 'air_quality')
    copy_file(city, 'AOI')
    copy_file(city, 'bu_proj_fu_ssp1_2050')
    copy_file(city, 'bu_proj_fu_ssp1_2100')
    copy_file(city, 'bu_proj_fu_ssp2_2050')
    copy_file(city, 'bu_proj_fu_ssp2_2100')
    copy_file(city, 'bu_proj_fu_ssp3_2050')
    copy_file(city, 'bu_proj_fu_ssp3_2100')
    copy_file(city, 'bu_proj_pu_ssp1_2050')
    copy_file(city, 'bu_proj_pu_ssp1_2100')
    copy_file(city, 'bu_proj_pu_ssp2_2050')
    copy_file(city, 'bu_proj_pu_ssp2_2100')
    copy_file(city, 'bu_proj_pu_ssp3_2050')
    copy_file(city, 'bu_proj_pu_ssp3_2100')
    copy_file(city, 'summer')
    copy_file(city, 'summer_day_ssp1')
    copy_file(city, 'summer_day_ssp2')
    copy_file(city, 'summer_day_ssp3')
    copy_file(city, 'summer_night_ssp1')
    copy_file(city, 'summer_night_ssp2')
    copy_file(city, 'summer_night_ssp3')
    copy_file(city, 'WSF_fu', 'fluvial', 'fluvial')
    copy_file(city, 'WSF_pu', 'pluvial', 'pluvial')
    
    city1 = city.lower().replace(' ', '_')
    
    if os.path.exists(Path(city) / 'data' / (city1 + '_landslide.tif')):
        copy_file(city, 'LS', 'landslide', 'landslide')
    
    for y in ['2050', '2100']:
        for s in ['', '_RL10']:
            if os.path.exists(Path(city) / 'data' / (city1 + '_slr' + s + '_' + y + '.tif')):
                copy_file(city, 'WSF_slr' + s + '_' + y, 'slr' + s + '_' + y, 'slr' + s + '_' + y)

### Share stats

In [5]:
create_folder('stats/fluvial')
create_folder('stats/pluvial')
for y in ['2050', '2100']:
    for s in ['', '_RL10']:
        create_folder(Path('stats') / ('slr' + s + '_' + y))

In [7]:
for city in cities:
    city1 = city.lower().replace(' ', '_')
    copyfile(Path(city) / 'stats' / 'wsf_fu.csv',
             Path('stats') / 'fluvial' / (city1 + '_fluvial.csv'))
    copyfile(Path(city) / 'stats' / 'wsf_pu.csv',
             Path('stats') / 'pluvial' / (city1 + '_pluvial.csv'))
    for y in ['2050', '2100']:
        for s in ['', '_RL10']:
            try:
                copyfile(Path(city) / 'stats' / ('slr' + s + '_' + y + '.csv'),
                         Path('stats') / ('slr' + s + '_' + y) / (city1 + '_slr' + s + '_' + y + '.csv'))
            except FileNotFoundError:
                pass

### Share data files

In [14]:
create_folder('share')
create_folder('share/total_water_storage_anomaly')
for city in cities:
    create_folder(Path('share') / city)
    create_folder(Path('share') / city / 'AOI')

In [9]:
for city in cities:
    city1 = city.lower().replace(' ', '_')
    # dictionary matching original raster file names and more readable file names that will be shared
    # keys: original raster file names
    # values: shareable file names
    share_list = {city1 + '_air_quality.tif': city1 + '_air_quality.tif',
                  city1 + '_landslide.tif': city1 + '_landslide.tif',
                  city1 + '_Summer.tif': city1 + '_summer_temp.tif',
                  city1 + '_WSF2019_4326.tif': city1 + '_settlement_2019.tif',
                  city1 + '_WSFevolution_4326.tif': city1 + '_settlement_by_year.tif',
                  'class1_fu.tif': city1 + '_fluvial_10pct.tif',
                  'class1_pu.tif': city1 + '_pluvial_10pct.tif',
                  'class2_fu.tif': city1 + '_fluvial_1-10pct.tif',
                  'class2_pu.tif': city1 + '_pluvial_1-10pct.tif',
                  'class3_fu.tif': city1 + '_fluvial_1pct.tif',
                  'class3_pu.tif': city1 + '_pluvial_1pct.tif',
                  city1 + '_bu_ssp1_2050.tif': city1 + '_built_up_projection_ssp1_2050.tif',
                  city1 + '_bu_ssp1_2100.tif': city1 + '_built_up_projection_ssp1_2100.tif',
                  city1 + '_bu_ssp2_2050.tif': city1 + '_built_up_projection_ssp2_2050.tif',
                  city1 + '_bu_ssp2_2100.tif': city1 + '_built_up_projection_ssp2_2100.tif',
                  city1 + '_bu_ssp3_2050.tif': city1 + '_built_up_projection_ssp3_2050.tif',
                  city1 + '_bu_ssp3_2100.tif': city1 + '_built_up_projection_ssp3_2100.tif',
                  city1 + '_lc.tif': city1 + '_landcover.tif',
                  city1 + '_slr_2050.tif': city1 + '_slr_2050.tif',
                  city1 + '_slr_2100.tif': city1 + '_slr_2100.tif',
                  city1 + '_slr_RL10_2050.tif': city1 + '_slr_10pct_flood_2050.tif',
                  city1 + '_slr_RL10_2100.tif': city1 + '_slr_10pct_flood_2100.tif',
                  city1 + '_urban-ssp1_day_sum.tif': city1 + '_summer_day_temp_projection_ssp1.tif',
                  city1 + '_urban-ssp1_nig_sum.tif': city1 + '_summer_night_temp_projection_ssp1.tif',
                  city1 + '_urban-ssp2_day_sum.tif': city1 + '_summer_day_temp_projection_ssp2.tif',
                  city1 + '_urban-ssp2_nig_sum.tif': city1 + '_summer_night_temp_projection_ssp2.tif',
                  city1 + '_urban-ssp3_day_sum.tif': city1 + '_summer_day_temp_projection_ssp3.tif',
                  city1 + '_urban-ssp3_nig_sum.tif': city1 + '_summer_night_temp_projection_ssp3.tif'}
    
    for file in share_list:
        try:
            copyfile(city / Path('output') / file, Path('share') / city / share_list[file])
        except FileNotFoundError:
            pass
    
    for shp_suf in ['cpg', 'dbf', 'prj', 'shp', 'shx']:
        copyfile(city / Path('output') / 'AOI' / (city1 + '_AOI.' + shp_suf),
                 Path('share') / city / 'AOI' / (city1 + '_AOI.' + shp_suf))
        copyfile(city / Path('output') / city / ('edges.' + shp_suf),
                 Path('share') / city / (city1 + '_roads.' + shp_suf))
        copyfile(city / Path('output') / city / ('nodes.' + shp_suf),
                 Path('share') / city / (city1 + '_intersections.' + shp_suf))
        copyfile(city / Path('output') / ('FB_QK_data_' + city1 + '.' + shp_suf),
                 Path('share') / city / (city1 + '_relative_wealth_index.' + shp_suf))
        
        try:
            for infra in ['health', 'schools']:
                copyfile(city / Path('output') / 'osm_infrastructure' / ('osm_' + infra + '.' + shp_suf),
                         Path('share') / city / (city1 + '_' + infra + '.' + shp_suf))
        except FileNotFoundError:
            pass
        
        try:
            for dist in ['1000', '2000', '3000']:
                copyfile(city / Path('output') / (city.replace(' ', '_') + '_AOI_accessibility_health_isochrones_' + dist + 'm.' + shp_suf),
                         Path('share') / city / (city.replace(' ', '_') + '_health_access_isochrones_' + dist + 'm.' + shp_suf))
        except FileNotFoundError:
            pass
        
        try:
            for dist in ['800', '1600', '2400']:
                copyfile(city / Path('output') / (city.replace(' ', '_') + '_AOI_accessibility_schools_isochrones_' + dist + 'm.' + shp_suf),
                         Path('share') / city / (city.replace(' ', '_') + '_schools_access_isochrones_' + dist + 'm.' + shp_suf))
        except FileNotFoundError:
            pass

In [13]:
city_coords = {'Battambang': (103.25, 13.25), 
               'Kampot': (104.25, 10.75),
               'Kep': (104.25, 10.75),
               'Poipet': (102.75, 13.75),
               'Siem Reap': (103.75, 13.25),
               'Sihanoukville': (103.75, 10.75),
               'Khemarak Phumin': (103.25, 11.75)}

for city in city_coords:
    city1 = city.lower().replace(' ', '_')
    for p in ['1', '12', '48']:
        copyfile(Path('data') / ('spei' + p) / ('_' + str(city_coords[city][0]) + '0000_  ' + str(city_coords[city][1]) + '0000.csv'),
                 Path('share') / city / (city1 + '_spei' + p + '.csv'))

In [20]:
for y in range(2011, 2021):
    for m in range(1, 13):
        copyfile(Path('output') / 'drought' / ('twsan_' + str(y) + str(m).zfill(2) + '.tif'),
                 Path('share') / 'total_water_storage_anomaly' / ('twsan_' + str(y) + str(m).zfill(2) + '.tif'))